### Parsing SeaBASS data from URL
Amir asked me how to parse seabass data. Below is one approach, that I think is general enough.
I use **requests** to get the data from a url
I use **re** to form regex queries to get field names (optionally, units) and figure out when the data parsing shoud start.
Data is read line by line and a dictionary (hash table) is populated with each datum appended to a list corresponding to fieldname key.
*Optionally* the dictionary is loaded onto a **pandas** data which makes data manipulation much easier. I use 3.5 but I believe this should work fine in 2.7.

In [1]:
import re
import requests
import pandas as pd # optional

In [2]:
url='http://seabass.gsfc.nasa.gov/seabass_archive/BIGELOW/BALCH/FERRY_00/2K/s000914w/main-s000914w.txt'

In [3]:
resp = requests.get(url)

In [4]:
stuff = resp.text.splitlines()

In [5]:
resp.close()

In [6]:
# Pre-compute some regex
columns = re.compile('^/fields=(.+)') # to get field/column names
units = re.compile('^/units=(.+)') # to get units -- optional
endHeader = re.compile('^/end_header') # to know when to start storing data

In [7]:
noFields = True
noUnits = True
getData = False
for line in stuff:
    if noFields:
        fieldStr = columns.findall(line)
        if len(fieldStr)>0:
            noFields = False
            fieldList = fieldStr[0].split(',')
            dataDict = dict.fromkeys(fieldList)
            continue # nothing left to do with this line
    if noUnits: # search for units in current line
        unitsStr = units.findall(line)
        if len(unitsStr) > 0:
            noUnits = False
            unitList = unitsStr[0].split(',')
            continue # nothing left to do with this line
    if not getData:
        if endHeader.match(line):
            getData = True
    else:
        dataList = line.split(' ')
        for field,datum in zip(fieldList,dataList):
            if not dataDict[field]:
                dataDict[field] = []
            dataDict[field].append(datum)

In [9]:
df = pd.DataFrame(dataDict,columns=fieldList)

In [10]:
df.head()

,year,month,day,hour,minute,second,lat,lon,Wt,sal,...,lt683.9,lsky412.7,lsky444.2,lsky488.9,lsky510.7,lsky554.7,lsky671.1,lsky683.5,senz,relaz
0,2000,9,14,14,6,58,43.77,-66.3495,13.2,31.88,...,7.20E-02,5.56E+00,4.82E+00,3.57E+00,2.91E+00,2.21E+00,9.20E-01,7.62E-01,140,119.27
1,2000,9,14,14,10,14,43.7681,-66.3727,13.7,32.23,...,7.03E-02,5.63E+00,4.88E+00,3.62E+00,2.97E+00,2.25E+00,9.68E-01,7.72E-01,140,118.384
2,2000,9,14,14,13,20,43.7662,-66.3938,13.65,32.2,...,7.15E-02,5.67E+00,4.92E+00,3.65E+00,3.03E+00,2.28E+00,1.02E+00,7.78E-01,140,116.894
3,2000,9,14,14,16,23,43.7641,-66.4145,14.37,32.26,...,6.73E-02,5.72E+00,4.96E+00,3.68E+00,3.10E+00,2.31E+00,1.07E+00,7.90E-01,140,115.158
4,2000,9,14,14,19,20,43.7626,-66.4351,14.69,32.31,...,6.45E-02,5.71E+00,4.94E+00,3.66E+00,3.06E+00,2.29E+00,1.02E+00,7.83E-01,140,116.57


In [11]:
df.shape

(152, 64)